In [2]:
import csv
import datetime

import pickle
from operator import sub
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn import preprocessing, ensemble

In [3]:
## 1. Data prep

mapping_dict = {
'ind_empleado'  : {-99:0, 'N':1, 'B':2, 'F':3, 'A':4, 'S':5},
'sexo'          : {'V':0, 'H':1, -99:2},
'ind_nuevo'     : {'0':0, '1':1, -99:2},
'indrel'        : {'1':0, '99':1, -99:2},
'indrel_1mes'   : {-99:0, '1.0':1, '1':1, '2.0':2, '2':2, '3.0':3, '3':3, '4.0':4, '4':4, 'P':5},
'tiprel_1mes'   : {-99:0, 'I':1, 'A':2, 'P':3, 'R':4, 'N':5},
'indresi'       : {-99:0, 'S':1, 'N':2},
'indext'        : {-99:0, 'S':1, 'N':2},
'conyuemp'      : {-99:0, 'S':1, 'N':2},
'indfall'       : {-99:0, 'S':1, 'N':2},
'tipodom'       : {-99:0, '1':1},
'ind_actividad_cliente' : {'0':0, '1':1, -99:2},
'segmento'      : {'02 - PARTICULARES':0, '03 - UNIVERSITARIO':1, '01 - TOP':2, -99:2},
'pais_residencia' : {'LV': 102, 'BE': 12, 'BG': 50, 'BA': 61, 'BM': 117, 'BO': 62, 'JP': 82, 'JM': 116, 'BR': 17, 'BY': 64, 'BZ': 113, 'RU': 43, 'RS': 89, 'RO': 41, 'GW': 99, 'GT': 44, 'GR': 39, 'GQ': 73, 'GE': 78, 'GB': 9, 'GA': 45, 'GN': 98, 'GM': 110, 'GI': 96, 'GH': 88, 'OM': 100, 'HR': 67, 'HU': 106, 'HK': 34, 'HN': 22, 'AD': 35, 'PR': 40, 'PT': 26, 'PY': 51, 'PA': 60, 'PE': 20, 'PK': 84, 'PH': 91, 'PL': 30, 'EE': 52, 'EG': 74, 'ZA': 75, 'EC': 19, 'AL': 25, 'VN': 90, 'ET': 54, 'ZW': 114, 'ES': 0, 'MD': 68, 'UY': 77, 'MM': 94, 'ML': 104, 'US': 15, 'MT': 118, 'MR': 48, 'UA': 49, 'MX': 16, 'IL': 42, 'FR': 8, 'MA': 38, 'FI': 23, 'NI': 33, 'NL': 7, 'NO': 46, 'NG': 83, 'NZ': 93, 'CI': 57, 'CH': 3, 'CO': 21, 'CN': 28, 'CM': 55, 'CL': 4, 'CA': 2, 'CG': 101, 'CF': 109, 'CD': 112, 'CZ': 36, 'CR': 32, 'CU': 72, 'KE': 65, 'KH': 95, 'SV': 53, 'SK': 69, 'KR': 87, 'KW': 92, 'SN': 47, 'SL': 97, 'KZ': 111, 'SA': 56, 'SG': 66, 'SE': 24, 'DO': 11, 'DJ': 115, 'DK': 76, 'DE': 10, 'DZ': 80, 'MK': 105, -99: 1, 'LB': 81, 'TW': 29, 'TR': 70, 'TN': 85, 'LT': 103, 'LU': 59, 'TH': 79, 'TG': 86, 'LY': 108, 'AE': 37, 'VE': 14, 'IS': 107, 'IT': 18, 'AO': 71, 'AR': 13, 'AU': 63, 'AT': 6, 'IN': 31, 'IE': 5, 'QA': 58, 'MZ': 27},
'canal_entrada' : {'013': 49, 'KHP': 160, 'KHQ': 157, 'KHR': 161, 'KHS': 162, 'KHK': 10, 'KHL': 0, 'KHM': 12, 'KHN': 21, 'KHO': 13, 'KHA': 22, 'KHC': 9, 'KHD': 2, 'KHE': 1, 'KHF': 19, '025': 159, 'KAC': 57, 'KAB': 28, 'KAA': 39, 'KAG': 26, 'KAF': 23, 'KAE': 30, 'KAD': 16, 'KAK': 51, 'KAJ': 41, 'KAI': 35, 'KAH': 31, 'KAO': 94, 'KAN': 110, 'KAM': 107, 'KAL': 74, 'KAS': 70, 'KAR': 32, 'KAQ': 37, 'KAP': 46, 'KAW': 76, 'KAV': 139, 'KAU': 142, 'KAT': 5, 'KAZ': 7, 'KAY': 54, 'KBJ': 133, 'KBH': 90, 'KBN': 122, 'KBO': 64, 'KBL': 88, 'KBM': 135, 'KBB': 131, 'KBF': 102, 'KBG': 17, 'KBD': 109, 'KBE': 119, 'KBZ': 67, 'KBX': 116, 'KBY': 111, 'KBR': 101, 'KBS': 118, 'KBP': 121, 'KBQ': 62, 'KBV': 100, 'KBW': 114, 'KBU': 55, 'KCE': 86, 'KCD': 85, 'KCG': 59, 'KCF': 105, 'KCA': 73, 'KCC': 29, 'KCB': 78, 'KCM': 82, 'KCL': 53, 'KCO': 104, 'KCN': 81, 'KCI': 65, 'KCH': 84, 'KCK': 52, 'KCJ': 156, 'KCU': 115, 'KCT': 112, 'KCV': 106, 'KCQ': 154, 'KCP': 129, 'KCS': 77, 'KCR': 153, 'KCX': 120, 'RED': 8, 'KDL': 158, 'KDM': 130, 'KDN': 151, 'KDO': 60, 'KDH': 14, 'KDI': 150, 'KDD': 113, 'KDE': 47, 'KDF': 127, 'KDG': 126, 'KDA': 63, 'KDB': 117, 'KDC': 75, 'KDX': 69, 'KDY': 61, 'KDZ': 99, 'KDT': 58, 'KDU': 79, 'KDV': 91, 'KDW': 132, 'KDP': 103, 'KDQ': 80, 'KDR': 56, 'KDS': 124, 'K00': 50, 'KEO': 96, 'KEN': 137, 'KEM': 155, 'KEL': 125, 'KEK': 145, 'KEJ': 95, 'KEI': 97, 'KEH': 15, 'KEG': 136, 'KEF': 128, 'KEE': 152, 'KED': 143, 'KEC': 66, 'KEB': 123, 'KEA': 89, 'KEZ': 108, 'KEY': 93, 'KEW': 98, 'KEV': 87, 'KEU': 72, 'KES': 68, 'KEQ': 138, -99: 6, 'KFV': 48, 'KFT': 92, 'KFU': 36, 'KFR': 144, 'KFS': 38, 'KFP': 40, 'KFF': 45, 'KFG': 27, 'KFD': 25, 'KFE': 148, 'KFB': 146, 'KFC': 4, 'KFA': 3, 'KFN': 42, 'KFL': 34, 'KFM': 141, 'KFJ': 33, 'KFK': 20, 'KFH': 140, 'KFI': 134, '007': 71, '004': 83, 'KGU': 149, 'KGW': 147, 'KGV': 43, 'KGY': 44, 'KGX': 24, 'KGC': 18, 'KGN': 11}
}
cat_cols = list(mapping_dict.keys())

target_cols = ['ind_ahor_fin_ult1','ind_aval_fin_ult1','ind_cco_fin_ult1','ind_cder_fin_ult1','ind_cno_fin_ult1','ind_ctju_fin_ult1','ind_ctma_fin_ult1','ind_ctop_fin_ult1','ind_ctpp_fin_ult1','ind_deco_fin_ult1','ind_deme_fin_ult1','ind_dela_fin_ult1','ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1','ind_plan_fin_ult1','ind_pres_fin_ult1','ind_reca_fin_ult1','ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1','ind_nomina_ult1','ind_nom_pens_ult1','ind_recibo_ult1']
target_cols = target_cols[2:]

In [4]:
# training data
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_ba6234822f384127b229f60fc9ccfe27 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='MZcFtwrNsiVZN593UTiDpPTY3cGsKFznqbWtB_D8CQqS',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_ba6234822f384127b229f60fc9ccfe27.get_object(Bucket='meraki-donotdelete-pr-o7utn8mfhdu9qq',Key='Meraki_Dataset.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

train_df = pd.read_csv(body)
train_df.head()


/opt/conda/envs/Python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,28-01-2015,1375586,N,ES,H,35.0,12-01-2015,0.0,6.0,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
1,28-01-2015,1050611,N,ES,V,23.0,10-08-2012,0.0,35.0,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
2,28-01-2015,1050612,N,ES,V,23.0,10-08-2012,0.0,35.0,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
3,28-01-2015,1050613,N,ES,H,22.0,10-08-2012,0.0,35.0,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
4,28-01-2015,1050614,N,ES,V,23.0,10-08-2012,0.0,35.0,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0


In [6]:
train_df.isnull().any()

fecha_dato               False
ncodpers                 False
ind_empleado              True
pais_residencia           True
sexo                      True
age                       True
fecha_alta                True
ind_nuevo                 True
antiguedad                True
indrel                    True
ult_fec_cli_1t            True
indrel_1mes               True
tiprel_1mes               True
indresi                   True
indext                    True
conyuemp                  True
canal_entrada             True
indfall                   True
tipodom                   True
cod_prov                  True
nomprov                   True
ind_actividad_cliente     True
renta                     True
segmento                  True
ind_ahor_fin_ult1        False
ind_aval_fin_ult1        False
ind_cco_fin_ult1         False
ind_cder_fin_ult1        False
ind_cno_fin_ult1         False
ind_ctju_fin_ult1        False
ind_ctma_fin_ult1        False
ind_ctop_fin_ult1        False
ind_ctpp

In [5]:
train_df.loc[train_df.age < 18,"age"]  = train_df.loc[(train_df.age >= 18) & (train_df.age <= 30),"age"].mean(skipna=True)
train_df.loc[train_df.age > 100,"age"] = train_df.loc[(train_df.age >= 30) & (train_df.age <= 100),"age"].mean(skipna=True)
train_df["age"].fillna(train_df["age"].mean(),inplace=True)
train_df["age"]                  = train_df["age"].astype(int)

In [53]:
train_df["ind_nuevo"].isnull().sum()

327

In [6]:
months_active = train_df.loc[train_df["ind_nuevo"].isnull(),:].groupby("ncodpers", sort=False).size()
months_active.max()

1

In [7]:
train_df.loc[train_df["ind_nuevo"].isnull(),"ind_nuevo"] = 1

In [56]:
train_df.head()

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,28-01-2015,1375586,N,ES,H,35,12-01-2015,0.0,6.0,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
1,28-01-2015,1050611,N,ES,V,23,10-08-2012,0.0,35.0,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
2,28-01-2015,1050612,N,ES,V,23,10-08-2012,0.0,35.0,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
3,28-01-2015,1050613,N,ES,H,22,10-08-2012,0.0,35.0,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0
4,28-01-2015,1050614,N,ES,V,23,10-08-2012,0.0,35.0,1.0,...,0,0,0,0,0,0,0,0.0,0.0,0


In [8]:
train_df.antiguedad = pd.to_numeric(train_df.antiguedad,errors="coerce")

In [14]:
train_df.antiguedad.isnull().sum()

327

In [9]:
train_df.loc[train_df.antiguedad.isnull(),"antiguedad"] = train_df.antiguedad.min()
train_df.loc[train_df.antiguedad <0, "antiguedad"]      = 0

In [10]:
dates=train_df.loc[:,"fecha_alta"].sort_values().reset_index()
median_date = int(np.median(dates.index.values))
train_df.loc[train_df.fecha_alta.isnull(),"fecha_alta"] = dates.loc[median_date,"fecha_alta"]
train_df["fecha_alta"].describe()

count          50000
unique           370
top       06-08-2012
freq            1431
Name: fecha_alta, dtype: object

In [11]:
train_df.loc[train_df.indrel.isnull(),"indrel"] = 1

In [12]:
train_df.drop(["tipodom","cod_prov"],axis=1,inplace=True)

In [19]:
train_df.isnull().any()

fecha_dato               False
ncodpers                 False
ind_empleado              True
pais_residencia           True
sexo                      True
age                      False
fecha_alta               False
ind_nuevo                False
antiguedad               False
indrel                   False
ult_fec_cli_1t            True
indrel_1mes               True
tiprel_1mes               True
indresi                   True
indext                    True
conyuemp                  True
canal_entrada             True
indfall                   True
nomprov                   True
ind_actividad_cliente     True
renta                     True
segmento                  True
ind_ahor_fin_ult1        False
ind_aval_fin_ult1        False
ind_cco_fin_ult1         False
ind_cder_fin_ult1        False
ind_cno_fin_ult1         False
ind_ctju_fin_ult1        False
ind_ctma_fin_ult1        False
ind_ctop_fin_ult1        False
ind_ctpp_fin_ult1        False
ind_deco_fin_ult1        False
ind_deme

In [13]:
train_df.loc[train_df.ind_actividad_cliente.isnull(),"ind_actividad_cliente"] = \
train_df["ind_actividad_cliente"].median()

In [14]:
train_df.nomprov.unique()

array(['MALAGA', 'CIUDAD REAL', 'ZARAGOZA', 'TOLEDO', 'LEON', 'GIPUZKOA',
       'CACERES', 'GIRONA', 'ZAMORA', 'BARCELONA', 'SALAMANCA', 'BURGOS',
       'HUESCA', 'NAVARRA', 'AVILA', 'SEGOVIA', 'LUGO', 'LERIDA',
       'MADRID', 'ALICANTE', 'SORIA', 'SEVILLA', 'CANTABRIA',
       'BALEARS, ILLES', 'VALLADOLID', 'PONTEVEDRA', 'VALENCIA', 'TERUEL',
       'CORUÑA, A', 'OURENSE', 'JAEN', 'CUENCA', 'BIZKAIA', 'CASTELLON',
       'RIOJA, LA', 'ALBACETE', 'BADAJOZ', 'MURCIA', 'CADIZ', nan,
       'ALMERIA', 'GUADALAJARA', 'PALENCIA', 'PALMAS, LAS', 'CORDOBA',
       'HUELVA', 'GRANADA', 'ASTURIAS', 'SANTA CRUZ DE TENERIFE',
       'MELILLA', 'TARRAGONA', 'ALAVA', 'CEUTA'], dtype=object)

In [15]:
train_df.loc[train_df.nomprov=="CORU\xc3\x91A, A","nomprov"] = "CORUNA, A"

In [16]:
train_df.loc[train_df.nomprov.isnull(),"nomprov"] = "UNKNOWN"

In [17]:
grouped        = train_df.groupby("nomprov").agg({"renta":lambda x: x.median(skipna=True)}).reset_index()
new_incomes    = pd.merge(train_df,grouped,how="inner",on="nomprov").loc[:, ["nomprov","renta_y"]]
new_incomes    = new_incomes.rename(columns={"renta_y":"renta"}).sort_values("renta").sort_values("nomprov")
train_df.sort_values("nomprov",inplace=True)
train_df             = train_df.reset_index()
new_incomes    = new_incomes.reset_index()

In [18]:
train_df.loc[train_df.renta.isnull(),"renta"] = new_incomes.loc[train_df.renta.isnull(),"renta"].reset_index()
train_df.loc[train_df.renta.isnull(),"renta"] = train_df.loc[train_df.renta.notnull(),"renta"].median()
train_df.sort_values(by="fecha_dato",inplace=True)

In [68]:
train_df.head()

,index,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,49587,28-01-2015,1124701,N,ES,V,39,15-03-2013,0.0,28.0,...,0,0,0,0,0,0,0,0.0,0.0,0
33328,16827,28-01-2015,1058107,N,ES,H,59,11-09-2012,0.0,34.0,...,0,0,0,0,0,0,0,0.0,0.0,0
33329,16819,28-01-2015,1058098,N,ES,V,71,11-09-2012,0.0,34.0,...,0,0,0,0,0,0,0,0.0,0.0,0
33330,48015,28-01-2015,1099883,N,ES,H,25,13-11-2012,0.0,32.0,...,0,0,0,0,0,0,0,0.0,0.0,0
33331,16815,28-01-2015,1058094,N,ES,H,33,11-09-2012,0.0,34.0,...,0,0,0,0,0,0,0,0.0,0.0,0


In [27]:
train_df.isnull().any()

index                    False
fecha_dato               False
ncodpers                 False
ind_empleado              True
pais_residencia           True
sexo                      True
age                      False
fecha_alta               False
ind_nuevo                False
antiguedad               False
indrel                   False
ult_fec_cli_1t            True
indrel_1mes               True
tiprel_1mes               True
indresi                   True
indext                    True
conyuemp                  True
canal_entrada             True
indfall                   True
nomprov                  False
ind_actividad_cliente    False
renta                    False
segmento                  True
ind_ahor_fin_ult1        False
ind_aval_fin_ult1        False
ind_cco_fin_ult1         False
ind_cder_fin_ult1        False
ind_cno_fin_ult1         False
ind_ctju_fin_ult1        False
ind_ctma_fin_ult1        False
ind_ctop_fin_ult1        False
ind_ctpp_fin_ult1        False
ind_deco

In [19]:
train_df.loc[train_df.ind_nomina_ult1.isnull(), "ind_nomina_ult1"] = 0
train_df.loc[train_df.ind_nom_pens_ult1.isnull(), "ind_nom_pens_ult1"] = 0

In [70]:
train_df.head()

,index,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,49587,28-01-2015,1124701,N,ES,V,39,15-03-2013,0.0,28.0,...,0,0,0,0,0,0,0,0.0,0.0,0
33328,16827,28-01-2015,1058107,N,ES,H,59,11-09-2012,0.0,34.0,...,0,0,0,0,0,0,0,0.0,0.0,0
33329,16819,28-01-2015,1058098,N,ES,V,71,11-09-2012,0.0,34.0,...,0,0,0,0,0,0,0,0.0,0.0,0
33330,48015,28-01-2015,1099883,N,ES,H,25,13-11-2012,0.0,32.0,...,0,0,0,0,0,0,0,0.0,0.0,0
33331,16815,28-01-2015,1058094,N,ES,H,33,11-09-2012,0.0,34.0,...,0,0,0,0,0,0,0,0.0,0.0,0


In [20]:
string_data = train_df.select_dtypes(include=["object"])
missing_columns = [col for col in string_data if string_data[col].isnull().any()]

In [72]:
train_df.isnull().any()

index                    False
fecha_dato               False
ncodpers                 False
ind_empleado              True
pais_residencia           True
sexo                      True
age                      False
fecha_alta               False
ind_nuevo                False
antiguedad               False
indrel                   False
ult_fec_cli_1t            True
indrel_1mes               True
tiprel_1mes               True
indresi                   True
indext                    True
conyuemp                  True
canal_entrada             True
indfall                   True
nomprov                  False
ind_actividad_cliente    False
renta                    False
segmento                  True
ind_ahor_fin_ult1        False
ind_aval_fin_ult1        False
ind_cco_fin_ult1         False
ind_cder_fin_ult1        False
ind_cno_fin_ult1         False
ind_ctju_fin_ult1        False
ind_ctma_fin_ult1        False
ind_ctop_fin_ult1        False
ind_ctpp_fin_ult1        False
ind_deco

In [43]:
train_df.loc[train_df.indfall.isnull(),"indfall"] = "N"
train_df.loc[train_df.tiprel_1mes.isnull(),"tiprel_1mes"] = "A"
# train_df.tiprel_1mes = train_df.tiprel_1mes.astype("category")

map_dict = { 1.0  : "1",
            "1.0" : "1",
            "1"   : "1",
            "3.0" : "3",
            "P"   : "P",
            3.0   : "3",
            2.0   : "2",
            "3"   : "3",
            "2.0" : "2",
            "4.0" : "4",
            "4"   : "4",
            "2"   : "2"}

train_df.indrel_1mes.fillna("P",inplace=True)
train_df.indrel_1mes = train_df.indrel_1mes.apply(lambda x: map_dict.get(x,x))
train_df.indrel_1mes = train_df.indrel_1mes.astype("category")


unknown_cols = [col for col in missing_columns if col not in ["indfall","tiprel_1mes","indrel_1mes"]]
for col in unknown_cols:
    train_df.loc[train_df[col].isnull(),col] = "UNKNOWN"

In [74]:
train_df.head()

,index,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,49587,28-01-2015,1124701,N,ES,V,39,15-03-2013,0.0,28.0,...,0,0,0,0,0,0,0,0.0,0.0,0
33328,16827,28-01-2015,1058107,N,ES,H,59,11-09-2012,0.0,34.0,...,0,0,0,0,0,0,0,0.0,0.0,0
33329,16819,28-01-2015,1058098,N,ES,V,71,11-09-2012,0.0,34.0,...,0,0,0,0,0,0,0,0.0,0.0,0
33330,48015,28-01-2015,1099883,N,ES,H,25,13-11-2012,0.0,32.0,...,0,0,0,0,0,0,0,0.0,0.0,0
33331,16815,28-01-2015,1058094,N,ES,H,33,11-09-2012,0.0,34.0,...,0,0,0,0,0,0,0,0.0,0.0,0


In [75]:
train_df.isnull().any()

index                    False
fecha_dato               False
ncodpers                 False
ind_empleado             False
pais_residencia          False
sexo                     False
age                      False
fecha_alta               False
ind_nuevo                False
antiguedad               False
indrel                   False
ult_fec_cli_1t           False
indrel_1mes              False
tiprel_1mes              False
indresi                  False
indext                   False
conyuemp                  True
canal_entrada            False
indfall                  False
nomprov                  False
ind_actividad_cliente    False
renta                    False
segmento                 False
ind_ahor_fin_ult1        False
ind_aval_fin_ult1        False
ind_cco_fin_ult1         False
ind_cder_fin_ult1        False
ind_cno_fin_ult1         False
ind_ctju_fin_ult1        False
ind_ctma_fin_ult1        False
ind_ctop_fin_ult1        False
ind_ctpp_fin_ult1        False
ind_deco

In [44]:
feature_cols = train_df.iloc[:1,].filter(regex="ind_+.*ult.*").columns.values
for col in feature_cols:
    train_df[col] = train_df[col].astype(int)

In [45]:
unique_months = pd.DataFrame(pd.Series(train_df.fecha_dato.unique()).sort_values()).reset_index(drop=True)
unique_months["month_id"] = pd.Series(range(1,1+unique_months.size)) # start with month 1, not 0 to match what we already have
unique_months["month_next_id"] = 1 + unique_months["month_id"]
unique_months.rename(columns={0:"fecha_dato"},inplace=True)
train_df = pd.merge(train_df,unique_months,on="fecha_dato")

In [24]:
train_df["sexo"].unique()

array(['V', 'H', 'UNKNOWN'], dtype=object)

In [46]:
train_df.loc[train_df["sexo"] == 'V', "sexo"] = 0
train_df.loc[train_df["sexo"] == 'H', "sexo"] = 1
train_df.loc[train_df["sexo"] == 'UNKNOWN', "sexo"] = 2

train_df["sexo"].unique()

array([0, 1, 2])

In [47]:
train_df["tiprel_1mes"].unique()

[A, I, P]
Categories (3, object): [A, I, P]

In [26]:
train_df.columns

Index(['index', 'fecha_dato', 'ncodpers', 'ind_empleado', 'pais_residencia',
       'sexo', 'age', 'fecha_alta', 'ind_nuevo', 'antiguedad', 'indrel',
       'ult_fec_cli_1t', 'indrel_1mes', 'tiprel_1mes', 'indresi', 'indext',
       'conyuemp', 'canal_entrada', 'indfall', 'nomprov',
       'ind_actividad_cliente', 'renta', 'segmento', 'ind_ahor_fin_ult1',
       'ind_aval_fin_ult1', 'ind_cco_fin_ult1', 'ind_cder_fin_ult1',
       'ind_cno_fin_ult1', 'ind_ctju_fin_ult1', 'ind_ctma_fin_ult1',
       'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1', 'ind_deco_fin_ult1',
       'ind_deme_fin_ult1', 'ind_dela_fin_ult1', 'ind_ecue_fin_ult1',
       'ind_fond_fin_ult1', 'ind_hip_fin_ult1', 'ind_plan_fin_ult1',
       'ind_pres_fin_ult1', 'ind_reca_fin_ult1', 'ind_tjcr_fin_ult1',
       'ind_valo_fin_ult1', 'ind_viv_fin_ult1', 'ind_nomina_ult1',
       'ind_nom_pens_ult1', 'ind_recibo_ult1', 'month_id', 'month_next_id'],
      dtype='object')

In [59]:
# XGB model

X_train = train_df.loc[1:, ['ncodpers', 
       'sexo', 'age', 'ind_nuevo', 'antiguedad', 'indrel',
       'ind_actividad_cliente', 'renta']]
print(X_train)

Y_train = train_df.loc[1:, target_cols]
print(Y_train)

       ncodpers  sexo  age  ind_nuevo  antiguedad  indrel  \
1       1058107     1   59        0.0        34.0     1.0   
2       1058098     0   71        0.0        34.0     1.0   
3       1099883     1   25        0.0        32.0     1.0   
4       1058094     1   33        0.0        34.0     1.0   
5       1058112     0   24        0.0        34.0     1.0   
6       1058116     1   23        0.0        34.0     1.0   
7       1099880     1   25        0.0        32.0     1.0   
8       1058134     1   24        0.0        34.0     1.0   
9       1099735     0   26        0.0        32.0     1.0   
10      1058110     0   23        0.0        34.0     1.0   
11      1099830     1   22        0.0        32.0     1.0   
12      1035437     0   24        0.0        36.0     1.0   
13      1056333     0   22        0.0        34.0     1.0   
14      1057587     0   27        0.0        34.0     1.0   
15      1056252     1   22        0.0        34.0     1.0   
16      1057620     0   

In [35]:
train_df.ind_empleado.unique()

array(['N', 'UNKNOWN'], dtype=object)

In [60]:
train_df.isin(['P']).any()

index                    False
fecha_dato               False
ncodpers                 False
ind_empleado             False
pais_residencia          False
sexo                     False
age                      False
fecha_alta               False
ind_nuevo                False
antiguedad               False
indrel                   False
ult_fec_cli_1t           False
indrel_1mes               True
tiprel_1mes               True
indresi                  False
indext                   False
conyuemp                 False
canal_entrada            False
indfall                  False
nomprov                  False
ind_actividad_cliente    False
renta                    False
segmento                 False
ind_ahor_fin_ult1        False
ind_aval_fin_ult1        False
ind_cco_fin_ult1         False
ind_cder_fin_ult1        False
ind_cno_fin_ult1         False
ind_ctju_fin_ult1        False
ind_ctma_fin_ult1        False
ind_ctop_fin_ult1        False
ind_ctpp_fin_ult1        False
ind_deco

In [61]:
print(X_train.shape)
print(Y_train.shape)

(49999, 8)
(49999, 22)


In [66]:
param = {}
param['objective'] = 'multi:softprob'
param['eta'] = 0.05
param['max_depth'] = 8
param['silent'] = 1
param['num_class'] = 22
param['eval_metric'] = "mlogloss"
param['min_child_weight'] = 1
param['subsample'] = 0.7
param['colsample_bytree'] = 0.7
param['seed'] = 0
num_rounds = 50

plst = list(param.items())

from sklearn.multioutput import MultiOutputRegressor

multi_output_regressor = MultiOutputRegressor(xgb.XGBRegressor(objective = 'multi:softprob', num_class = 22, max_depth = 8, colsample_bytree = 0.7, seed = 0))
xgb_multi_model = multi_output_regressor.fit(X_train, Y_train)

In [67]:
print(xgb_multi_model)

MultiOutputRegressor(estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.7, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=8, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, num_class=22, objective='multi:softprob',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=0, silent=True, subsample=1),
           n_jobs=None)


In [86]:
# test
body = client_ba6234822f384127b229f60fc9ccfe27.get_object(Bucket='meraki-donotdelete-pr-o7utn8mfhdu9qq',Key='Meraki_Test.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

test_df = pd.read_csv(body)
test_df.head()


,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento
0,28-06-2016,15889,F,ES,V,56,16-01-1995,0,256,1,...,N,N,KAT,N,1,28,MADRID,1,326124.90,01 - TOP
1,28-06-2016,1170544,N,ES,H,36,28-08-2013,0,34,1,...,N,NaN,KAT,N,1,3,ALICANTE,0,NA,02 - PARTICULARES
2,28-06-2016,1170545,N,ES,V,22,28-08-2013,0,34,1,...,N,NaN,KHE,N,1,15,"CORUÑA, A",1,NA,03 - UNIVERSITARIO
3,28-06-2016,1170547,N,ES,H,22,28-08-2013,0,34,1,...,N,NaN,KHE,N,1,8,BARCELONA,0,148402.98,03 - UNIVERSITARIO
4,28-06-2016,1170548,N,ES,H,22,28-08-2013,0,34,1,...,N,NaN,KHE,N,1,7,"BALEARS, ILLES",0,106885.80,03 - UNIVERSITARIO


In [75]:
test_df.isnull().any()

fecha_dato               False
ncodpers                 False
ind_empleado             False
pais_residencia          False
sexo                     False
age                      False
fecha_alta               False
ind_nuevo                False
antiguedad               False
indrel                   False
ult_fec_cli_1t            True
indrel_1mes              False
tiprel_1mes              False
indresi                  False
indext                   False
conyuemp                  True
canal_entrada            False
indfall                  False
tipodom                  False
cod_prov                 False
nomprov                  False
ind_actividad_cliente    False
renta                    False
segmento                 False
dtype: bool

In [87]:
test_df.drop(columns=['ult_fec_cli_1t', 'conyuemp'], axis = 1, inplace = True)

In [91]:
test_df.loc[test_df["sexo"] == 'V', "sexo"] = 0
test_df.loc[test_df["sexo"] == 'H', "sexo"] = 1
test_df.loc[test_df["sexo"] == 'UNKNOWN', "sexo"] = 2

In [95]:
test_df.loc[test_df["renta"] == '         NA', "renta"] = 0.0

In [96]:
X_test = test_df.loc[1:, ['ncodpers', 
       'sexo', 'age', 'ind_nuevo', 'antiguedad', 'indrel',
       'ind_actividad_cliente', 'renta']]

print(X_test.shape)

(499, 8)


In [97]:
print(X_test)

     ncodpers  sexo  age  ind_nuevo  antiguedad  indrel  \
1     1170544     1   36          0          34       1   
2     1170545     0   22          0          34       1   
3     1170547     1   22          0          34       1   
4     1170548     1   22          0          34       1   
5     1170550     0   22          0          34       1   
6     1170552     1   51          0          34       1   
7     1170553     1   22          0          34       1   
8     1170555     0   22          0          34       1   
9     1170557     1   22          0          34       1   
10    1170559     1   22          0          34       1   
11    1170563     1   22          0          34       1   
12    1170542     1   22          0          34       1   
13    1170565     1   22          0          34       1   
14    1170568     1   22          0          34       1   
15    1170570     1   22          0          34       1   
16    1170576     0   22          0          34       1 

In [98]:
Y_pred = xgb_multi_model.predict(X_test)

print(Y_pred)

[[[1.67997163e-02 9.99868751e-01 9.97076988e-01 ... 9.94372845e-01
   9.95503485e-01 9.94806349e-01]
  [9.63679180e-02 9.99868751e-01 9.92214859e-01 ... 9.87945974e-01
   9.92829144e-01 9.65747654e-01]
  [2.02760845e-02 9.99868751e-01 9.90241349e-01 ... 9.94135082e-01
   9.87780213e-01 9.95990932e-01]
  ...
  [8.33830424e-03 9.99868751e-01 9.95784342e-01 ... 9.98721063e-01
   9.98821557e-01 9.96862292e-01]
  [1.56406146e-02 9.99868751e-01 9.94850099e-01 ... 9.97422934e-01
   9.97373462e-01 9.97505128e-01]
  [1.62596665e-02 9.99868751e-01 9.95472968e-01 ... 9.98592436e-01
   9.98587132e-01 9.96569753e-01]]

 [[9.80972648e-01 6.24695031e-06 2.20985757e-03 ... 3.95193323e-03
   3.14182648e-03 3.87286535e-03]
  [9.01631653e-01 6.24695031e-06 6.76422659e-03 ... 1.03887999e-02
   5.66870719e-03 3.15897539e-02]
  [9.78692353e-01 6.24695031e-06 8.90862383e-03 ... 4.74585639e-03
   1.07074203e-02 3.39679979e-03]
  ...
  [9.90776777e-01 6.24695031e-06 3.57466354e-03 ... 7.12079869e-04
   6.78284

In [99]:
print(Y_pred.shape)

(22, 499, 22)


In [102]:
X_test = test_df.loc[1:2, ['ncodpers', 
       'sexo', 'age', 'ind_nuevo', 'antiguedad', 'indrel',
       'ind_actividad_cliente', 'renta']]

print(X_test.shape)

(2, 8)


In [103]:
Y_pred = xgb_multi_model.predict(X_test)

print(Y_pred)

[[[1.67997163e-02 9.99868751e-01 9.97076988e-01 9.99625087e-01
   9.99513030e-01 9.99868751e-01 9.99868751e-01 9.99725044e-01
   9.99653578e-01 9.98133957e-01 9.55828249e-01 9.99410272e-01
   9.99846816e-01 9.99542117e-01 9.96029735e-01 9.97662187e-01
   9.97750819e-01 9.99243140e-01 9.99865532e-01 9.94372845e-01
   9.95503485e-01 9.94806349e-01]
  [9.63679180e-02 9.99868751e-01 9.92214859e-01 9.98688042e-01
   9.99799550e-01 9.99868751e-01 9.99868751e-01 9.99728382e-01
   9.99704301e-01 9.96897221e-01 9.57523406e-01 9.99442756e-01
   9.99858260e-01 9.99182522e-01 9.98174906e-01 9.91165936e-01
   9.97771144e-01 9.99511600e-01 9.99866486e-01 9.87945974e-01
   9.92829144e-01 9.65747654e-01]]

 [[9.80972648e-01 6.24695031e-06 2.20985757e-03 6.60520818e-05
   1.45744547e-04 6.24695031e-06 6.24695031e-06 5.05447279e-05
   1.20559489e-04 1.25961390e-03 4.08469476e-02 2.94788886e-04
   1.36959079e-05 1.35568596e-04 2.77707586e-03 1.71285146e-03
   1.64397852e-03 3.29744391e-04 9.24757387e-06 

In [104]:
print(Y_pred.shape)

(22, 2, 22)
